In [88]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

from ASCE import load_combos

from steel_design.AISC import aisc
steel_density = 490 / 12**3 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

from wood_design.sawn_lumber.ref_design_values.table_4a import table_4a_load
from wood_design.sawn_lumber.ref_design_values.C_F import C_F_load
table_4a = table_4a_load(species="DF")

<div style="page-break-after: always;"></div>

# Seismic Load - Equivalent Lateral Force Method

### Seismic Parameters

In [ ]:
site_class = "default" #A, B, C, D, E, F, or default
S_s = 1.5
S_1 = 0.683

F_a = 1.2
F_v = "na"

In [89]:
S_DS = 0.203
S_D1 = 0.179
T_L = 8 #sec
I_e = 1.0

R = 6.5
Ω_0 = 3
C_d = 4
ρ = 1.0

### Seismic Response Coefficient, $C_s$

<img src="./attachments/Table_12.8-2.png" width=500>

In [90]:
hn = 16 #ft
Ct = 0.02
x = 0.75
T = Ct*hn**x

f"Approximate fundamental period, Ta = {round(T, 3)} sec"

'Approximate fundamental period, Ta = 0.16 sec'

In [91]:
Cs_calc = S_DS / (R/I_e)

if T <= T_L:
	Cs = min(Cs_calc, S_D1 / (T*R/I_e))
elif T > T_L:
	Cs = min(Cs_calc, S_D1 / (T**2*R/I_e))

f"Cs = {round(Cs, 4)}"

'Cs = 0.0312'

### Effective Seismic Weight

In [92]:
roof_DL = 15 #psf
wall_2x4_DL = 11 #psf
wall_2x6_DL = 8 #psf
cladded_wall_DL = 20 #psf

roof_area = 9660 #sf
wall_2x6_perimeter = 506 #ft
wall_2x4_perimeter = 391 #ft
wall_cladded_perimeter = 211 #ft

W = (
	roof_DL*roof_area \
	+ wall_2x4_DL*wall_2x4_perimeter \
	+ wall_2x6_DL*wall_2x6_perimeter \
	+ cladded_wall_DL*wall_cladded_perimeter
) / 1000 #kips

f"total seismic weight = {round(W / 1000, 2)} kips"

'total seismic weight = 0.16 kips'

### Base Shear

In [93]:
V = Cs * W #kips
f"Base Shear, V = {round(V, 2)} kips"

'Base Shear, V = 4.92 kips'

### Vertical Distribution

<img src="./attachments/vertical distribution.png" width=500>

In [83]:
h_roof = hn #ft
k = 1.0

Cv_roof = 1.0

F_roof = Cv_roof * V #kips
f"force on roof = {round(F_roof, 2)} kips"

'force on roof = 4.92 kips'

### Diaphragm Forces

In [84]:
F_px_calc = F_roof #kips
F_design_roof = F_px_calc #kips
f"Diaphragm Force = {round(F_design_roof, 2)} kips"

'Diaphragm Force = 4.92 kips'

### Story Drift

In [85]:
def seismic_drift(δ_xe):
	δ_x = C_d * δ_xe / I_e
	return δ_x

In [87]:
elastic_story_drifts = [0.1, 0.3, 5] #in (from external calculation)

seismic_story_drifts = []
for δ_xe in elastic_story_drifts:
	δ_x = seismic_drift(δ_xe)
	seismic_story_drifts.append(δ_x)
	
f"Seismic Drifts = {seismic_story_drifts} in"

'Seismic Drifts = [0.4, 1.2, 20.0] in'